In [7]:
import numpy as np

from smoothcon import SmoothCon

In [8]:
rng = np.random.default_rng(seed=1)

n = 100
x = rng.uniform(-2.0, 2.0, size=n)
y = x + rng.normal(loc=0.0, scale=1.0, size=n)
mcycle = {"accel": y, "times": x}  # imitating the MASS:mcycle dataset

In [9]:
smooth = SmoothCon(
    spec="s(times, bs='ps', k=20, m=c(3,2))",
    data=mcycle,
    knots=None,
    absorb_cons=True,
    diagonal_penalty=True,
    pass_to_r=None,
)

In [10]:
basis = smooth.basis
penalty = smooth.penalty

In [11]:
bases = smooth.all_bases()
penalties = smooth.all_penalties()

In [12]:
times_grid = np.linspace(mcycle["times"].min(), mcycle["times"].max(), 100)
prediction_basis = smooth.predict({"times": times_grid})

In [13]:
prediction_bases = smooth.predict_all_bases({"times": times_grid})

In [14]:
smooth = SmoothCon(
    spec="ti(times, accel, bs='ps')",
    data=mcycle,
    knots=None,
    absorb_cons=True,
    diagonal_penalty=True,
    pass_to_r=None,
)

In [15]:
# smooth.penalty # this will raise an error

In [16]:
penalties = smooth.all_penalties()[0]
S1 = penalties[0]